In [9]:
import pandas as pd
import numpy as np
from lxml import etree
import xml.etree.ElementTree as ET
import time
import math

In [ ]:
xml_positions = etree.parse('C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-003BEU.xml') ##arbeitsplatop

In [ ]:
kpimerged = pd.read_excel('C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/DFB_KPI_Merged.xlsx') #arbeit

In [ ]:
path='C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/'    #arbeit

In [1]:
path='C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/'    #Privat

In [6]:
xml_positions = etree.parse('C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-003BEU.xml') ##privat
kpimerged = pd.read_excel('C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/DFB_KPI_Merged.xlsx') ##privat

In [99]:
kpimergednew = kpimerged.rename({'FRAME_NUMBER': 'N'}, axis=1)

In [5]:
class DataReader:
  """
  Main class to read in xml files provided by the DFB.

  Returns:
      ElementTree root: The root of the xml document.

  """

  def __init__(self,xml_file):
    self.xml_root = self._load_data(xml_file)


  def _load_data(self,xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root

  def create_dataframe(self,df_cols=None,findall_string=".//Object",time_search=None):
    """
    Create a dataframe from the xml inputs

    Args:
        df_cols (list): Optional, the column names. If no names given, columns
          detected automatically
        findall_string (str): The xpath string to iterate trees (https://www.w3schools.com/xml/xml_xpath.asp)
        time_search (float): Will currently look at parent if it's an Event, it
          will compare the time_search to the EventTime. I.e. this is a means of getting
          all passes before the frame time.

    Returns:
        pd.dataframe
    """

    populate_column_names=False
    if df_cols is None:
        df_cols = []
        populate_column_names=True

    out_df = pd.DataFrame(columns = df_cols)

    parent_map = dict((c, p) for p in self.xml_root.getiterator() for c in p)
    for node in self.xml_root.findall(findall_string):
        if time_search is not None:
          if (parent_map[node].tag != "Event"): continue
          event_time = parent_map[node].attrib["EventTime"]
          ts = ciso8601.parse_datetime(event_time)
          # to get time in seconds:
          seconds=time.mktime(ts.timetuple())
          # If timestamp is larger than the search, exit
          if seconds > time_search: continue

        # If column names are not provided, get them
        if populate_column_names==True:
            for key,value in node.attrib.items():
                df_cols.append(key)
                populate_column_names=False

        res = []
        for c in df_cols:
            res.append(node.attrib.get(c))
        out_df = out_df.append(pd.Series(res, index = df_cols), ignore_index=True)
    return out_df


In [4]:
def get_tracking_data(xml_positions, half='firstHalf'):
        """Returns a dataframe with the tracking (position: x, y, z) of all players + the ball
        
        Dropping rows where ball positions is NAN

        +-------+-----------+-----------+
        |       |   BALL    |  TeamId   |
        +       +-----------+-----------+
        |       |  BallId   | PlayerId  |
        + N | T +---+---+---+---+---+---+
        |       | X | Y | Z | X | Y | A |
        +=======+===+===+===+===+===+===+
        | 0 | 0 |123|456|789|012|456|789|
        +-------+-----------+-----------+

        :param xml_positions: the parsed xml tracking data (lxml object)
        :param half: the description of the half: either "firstHalf" or "secondHalf", etc.
        :return: a dataframe containing the tracking data: position of all players and the ball
        """
        def change_type(df, frameset):
            """
            change type of columns
            different columns names and tpyes for the ball and for the rest of the players
            """
            if frameset.attrib.get('TeamId') == 'BALL':
                return df.astype(
                    # change types of columns for the ball
                    dtype={
                        'A':float, 
                        'D':float, 
                        'M':int, 
                        'N':int, 
                        'S':float, 
                        'X':float, 
                        'Y':float,
                        'Z':float,
                        #'T':'datetime64[ns]', # this breaks the timezone
                        'BallPossession':int, 
                        'BallStatus':int,
                    }, 
                    errors='raise'
                )
            else:
                return df.astype(
                    # change types of columns for the players
                    dtype={
                        'A':float, 
                        'D':float, 
                        'M':int, 
                        'N':int, 
                        'S':float, 
                        'X':float, 
                        'Y':float,
                        #'T':'datetime64[ns]', # this breaks the timezone
                    }, 
                    errors='raise'
                )
        

        def create_columns(df, frameset):
            """A utility function to create a multiindex columns"""
            person_id = frameset.attrib.get('PersonId')
            team_id = frameset.attrib.get('TeamId')
            df.columns = pd.MultiIndex.from_product(
                [[team_id], [person_id], df.columns],
                names=['TeamId', 'PersonId', 'Position']
            )
            return df

        tracking_players = pd.concat([
            pd.DataFrame.from_records(
                [dict(frame.attrib) for frame in frameset.getchildren()]
            ).pipe(change_type, frameset=frameset)
             .set_index(['N', 'T'])
             .pipe(create_columns, frameset=frameset)
            for frameset in xml_positions.xpath(F'//Positions/FrameSet[@GameSection = "{half}"]')
        ], axis=1, sort=False)
        
        # this is casting the 'T' values in the index to datetime with the correct time zone
        tracking_players.index.set_levels(pd.to_datetime(tracking_players.index.get_level_values(1)), level=1, inplace = True)

        return tracking_players

In [8]:
positions=get_tracking_data(xml_positions)     
##create dataframe from xml input

In [10]:
def calculateDistance(x1,y1,x2,y2):  
     dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
     return dist  

In [11]:
print (calculateDistance(32.4,15.3,48,12.5))

15.849290204927161


In [75]:
CUID1 = 'DFL-CLU-000N8Y'
cuid1_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K3_player (1).xml".format(CUID1)
cuid1_team_data = DataReader(path+cuid1_player_info)
cuid1_team_meta_df = cuid1_team_data.create_dataframe()
cuid1_player_IDs = cuid1_team_meta_df.ObjectId.tolist()

In [77]:
CUID2 = 'DFL-CLU-000N8Z'
cuid2_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K3_player (1).xml".format(CUID2)
cuid2_team_data = DataReader(path+cuid2_player_info)
cuid2_team_meta_df = cuid2_team_data.create_dataframe()
cuid2_player_IDs = cuid2_team_meta_df.ObjectId.tolist()

In [19]:
Germany=positions.loc[:,('DFL-CLU-000N8Y',slice(None),['X','Y'])] 

In [20]:
german_players = Germany.columns.get_level_values(level=1).unique()

In [24]:
Germany['DFL-CLU-000N8Y',german_players[0],'DtB']=0

C:\Users\Tim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [68]:
('DFL-OBJ-0000I4') in positions.columns.get_level_values(level=1)

True

In [40]:
i=3
calculateDistance(Germany['DFL-CLU-000N8Y','DFL-OBJ-000191','X'][i],Germany['DFL-CLU-000N8Y','DFL-OBJ-000191','Y'][i],positions['BALL','DFL-OBJ-0000XT','X'][i],positions['BALL','DFL-OBJ-0000XT','Y'][i])

9.950778863988488

In [81]:
players=positions.columns.get_level_values(level=1).unique()

In [93]:
positions[CUID1]['DFL-OBJ-000191']

,Position,X,Y,S,D,A,M
N,T,,,,,,
10000,2019-06-11 20:47:01.600000+02:00,-8.60,-4.81,0.00,0.00,0.00,1
10001,2019-06-11 20:47:01.640000+02:00,-8.59,-4.80,1.25,1.38,-0.05,1
10002,2019-06-11 20:47:01.680000+02:00,-8.59,-4.79,1.25,1.55,0.00,1
10003,2019-06-11 20:47:01.720000+02:00,-8.58,-4.78,1.25,1.55,0.06,1
10004,2019-06-11 20:47:01.760000+02:00,-8.57,-4.77,1.27,1.55,0.13,1
...,...,...,...,...,...,...,...
77670,2019-06-11 21:32:08.400000+02:00,-23.55,-2.93,16.86,18.34,0.00,46
77671,2019-06-11 21:32:08.440000+02:00,-23.37,-2.91,17.00,0.00,0.00,46
77672,2019-06-11 21:32:08.480000+02:00,-23.18,-2.90,17.01,0.00,0.00,46


In [89]:
players=players.delete(0)  #deletes the ball from the list of players

In [80]:
('DFL-OBJ-002FZY') in cuid2_player_IDs

True

In [103]:
half={} # frames der kickoff zeiten in frames
kickoffs=kpimergednew[(kpimergednew['SUBTYPE']=='Kickoff')].index.tolist()
for i in kickoffs:
    if i != 0:
        framedif=kpimergednew.iloc[i].N -kpimergednew.iloc[i-2].N
        if framedif > 10000:
            half['2nd half']=kpimergednew.iloc[i].N
    else:
        half['1st half']=kpimergednew.iloc[i].N

In [104]:
half

{'1st half': 10008, '2nd half': 100046}

In [142]:
i=1
lol=str(i)+ '. Substitution ' + CUID1
lol

'1. Substitution DFL-CLU-000N8Y'

In [143]:
substitutions={}
subs=kpimergednew[kpimergednew.SUBTYPE=='Substitution']
i=1
j=1
for index,rows in subs.iterrows():
    if subs.loc[index,'PUID1'] in cuid1_player_IDs:
        substitutions[str(i)+ '. Substitution ' + CUID1]=subs.loc[index,'PUID1']
        i=i+1
    else:
        substitutions[str(j)+ '. Substitution ' + CUID2]=subs.loc[index,'PUID1']
        j=j+1
        

In [148]:
substitutions

{'1. Substitution DFL-CLU-000N8Y': 'DFL-OBJ-0000SE',
 '2. Substitution DFL-CLU-000N8Y': 'DFL-OBJ-0000OJ',
 '1. Substitution DFL-CLU-000N8Z': 'DFL-OBJ-002G05',
 '3. Substitution DFL-CLU-000N8Y': 'DFL-OBJ-0000O3',
 '2. Substitution DFL-CLU-000N8Z': 'DFL-OBJ-002FZW',
 '3. Substitution DFL-CLU-000N8Z': 'DFL-OBJ-002FZZ'}

In [129]:
subs.loc[:,'PUID1']

889     DFL-OBJ-0000SE
1023    DFL-OBJ-0000OJ
1141    DFL-OBJ-002G05
1266    DFL-OBJ-0000O3
1363    DFL-OBJ-002FZW
1547    DFL-OBJ-002FZZ
Name: PUID1, dtype: object

In [94]:
dtb=[]
for player in players:
    if player in cuid1_player_IDs:
        for index,row in positions.iterrows():
            dtb.append(calculateDistance(positions[CUID1,player,'X'][index],positions[CUID1,player,'Y'][index],positions['BALL','DFL-OBJ-0000XT','X'][index],positions['BALL','DFL-OBJ-0000XT','Y'][index]))
        positions[CUID1][player]['DtB']=dtb    
    if player in cuid2_player_IDs: 
        for index,row in positions.iterrows():
            dtb.append(calculateDistance(positions[CUID2,player,'X'][index],positions[CUID2,player,'Y'][index],positions['BALL','DFL-OBJ-0000XT','X'][index],positions['BALL','DFL-OBJ-0000XT','Y'][index]))
        positions[CUID1][player]['DtB']=dtb

ValueError: Length of values does not match length of index

In [63]:
dtb

[9.98323093993122,
 9.969538605171254,
 9.964466869833027,
 9.950778863988488,
 9.93709212999457,
 9.923406673113824,
 9.909722498637386,
 9.901095898939671,
 9.3835227926403,
 8.879735356416878,
 8.386018125427587,
 7.891108920804477,
 7.3838878648040165,
 6.894200461257273,
 6.383290687411941,
 5.885108325256215,
 5.391892061234164,
 4.948949383455037,
 4.537929043076808,
 4.0989754817515065,
 3.6831779756074785,
 3.2841589486503238,
 2.890674661735561,
 2.5094421690885804,
 2.1421951358361357,
 1.7799157283422165,
 1.4312581877495052,
 1.0833743581975719,
 0.7502666192761078,
 0.4118252056394804,
 0.590338885725819,
 0.537121960079831,
 0.4939635614091386,
 0.4720169488482379,
 0.4666904755831208,
 0.07071067811865513,
 0.1341640786499869,
 0.1341640786499865,
 0.16124515496597208,
 0.10630145812734673,
 0.12806248474865675,
 0.1341640786499877,
 0.31256999216175607,
 0.17720045146669364,
 0.3059411708155668,
 0.4219004621945796,
 0.9002777349240624,
 1.366162508635045,
 1.829371476

In [32]:
Germany['DFL-CLU-000N8Y','DFL-OBJ-000191','X']

N      T                               
10000  2019-06-11 20:47:01.600000+02:00    -8.60
10001  2019-06-11 20:47:01.640000+02:00    -8.59
10002  2019-06-11 20:47:01.680000+02:00    -8.59
10003  2019-06-11 20:47:01.720000+02:00    -8.58
10004  2019-06-11 20:47:01.760000+02:00    -8.57
                                           ...  
77670  2019-06-11 21:32:08.400000+02:00   -23.55
77671  2019-06-11 21:32:08.440000+02:00   -23.37
77672  2019-06-11 21:32:08.480000+02:00   -23.18
77673  2019-06-11 21:32:08.520000+02:00   -22.99
77674  2019-06-11 21:32:08.560000+02:00   -22.80
Name: (DFL-CLU-000N8Y, DFL-OBJ-000191, X), Length: 67675, dtype: float64

In [17]:
for player in Germany:
    Germany

TeamId                                 DFL-CLU-000N8Y                       \
PersonId                               DFL-OBJ-000191       DFL-OBJ-0027G6   
Position                                            X     Y              X   
N     T                                                                      
10000 2019-06-11 20:47:01.600000+02:00          -8.60 -4.81           0.18   
10001 2019-06-11 20:47:01.640000+02:00          -8.59 -4.80           0.18   
10002 2019-06-11 20:47:01.680000+02:00          -8.59 -4.79           0.18   
10003 2019-06-11 20:47:01.720000+02:00          -8.58 -4.78           0.19   
10004 2019-06-11 20:47:01.760000+02:00          -8.57 -4.77           0.19   

TeamId                                                             \
PersonId                                     DFL-OBJ-0000I4         
Position                                   Y              X     Y   
N     T                                                             
10000 2019-06-11 20:47:01.600000+02:00  0.01         -47.11 -0.26   
10001 2019-06-11 20:47:01.640000+02:00  0.01         -47.06 -0.23   
10002 2019-06-11 20:47:01.680000+02:00  0.01         -47.01 -0.19   
10003 2019-06-11 20:47:01.720000+02:00  0.00         -46.95 -0.17   
10004 2019-06-11 20:47:01.760000+02:00  0.00         -46.91 -0.14   

TeamId                                                                       \
PersonId                               DFL-OBJ-0000SE        DFL-OBJ-0001HT   
Position                                            X      Y              X   
N     T                                                                       
10000 2019-06-11 20:47:01.600000+02:00         -13.14  16.98         -21.19   
10001 2019-06-11 20:47:01.640000+02:00         -13.13  16.96         -21.19   
10002 2019-06-11 20:47:01.680000+02:00         -13.13  16.92         -21.19   
10003 2019-06-11 20:47:01.720000+02:00         -13.12  16.90         -21.19   
10004 2019-06-11 20:47:01.760000+02:00         -13.12  16.86         -21.19   

TeamId                                        ...                       \
PersonId                                      ... DFL-OBJ-0000OJ         
Position                                   Y  ...              X     Y   
N     T                                       ...                        
10000 2019-06-11 20:47:01.600000+02:00  8.91  ...          -9.31  4.89   
10001 2019-06-11 20:47:01.640000+02:00  8.91  ...          -9.30  4.91   
10002 2019-06-11 20:47:01.680000+02:00  8.91  ...          -9.30  4.93   
10003 2019-06-11 20:47:01.720000+02:00  8.91  ...          -9.30  4.95   
10004 2019-06-11 20:47:01.760000+02:00  8.91  ...          -9.31  4.96   

TeamId                                                                       \
PersonId                               DFL-OBJ-0000ZS        DFL-OBJ-0002F5   
Position                                            X      Y              X   
N     T                                                                       
10000 2019-06-11 20:47:01.600000+02:00         -18.07 -10.54          -9.97   
10001 2019-06-11 20:47:01.640000+02:00         -18.07 -10.57          -9.97   
10002 2019-06-11 20:47:01.680000+02:00         -18.08 -10.59          -9.97   
10003 2019-06-11 20:47:01.720000+02:00         -18.08 -10.61          -9.97   
10004 2019-06-11 20:47:01.760000+02:00         -18.09 -10.62          -9.98   

TeamId                                                              \
PersonId                                     DFL-OBJ-0000O3          
Position                                   Y              X      Y   
N     T                                                              
10000 2019-06-11 20:47:01.600000+02:00  0.62          -0.01  11.47   
10001 2019-06-11 20:47:01.640000+02:00  0.62           0.02  11.47   
10002 2019-06-11 20:47:01.680000+02:00  0.63           0.07  11.47   
10003 2019-06-11 20:47:01.720000+02:00  0.63           0.11  11.48   
10004 2019-06-11 20:47:0